# Internal Networking in Kubernetes

### Introduction

Now so far, we have learned how to launch different pods, and groups of pods in deployments.  However we have not learned how to communicate with those pods.  For example, how would our frontend pod be able to connect to one of the backend pods. That's what we'll discuss in this lesson.

### Setting it Up

In previous lessons, we first created a **pod**, and then we learned how to create a **deployment** to manage multiple replicas of a pod.  

Now let's start creating a collection of frontend pods and then get them to communicate with our backend pods.  To start, of course, we'll just create another deployment for our frontend service -- that part isn't so difficult.  

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: frontend-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: frontend
  template:
    metadata:
      labels:
        component: frontend
    spec:
      containers:
        - name: restaurants-frontend
          image: jek2141/restaurants_frontend
          ports:
          - containerPort: 8501
```

But how do we connect our front end application with our backend api?  

### Networking through ip addresses

When working in AWS, we could do this simply by telling each application of the other's ip address.  So if we wanted the frontend application to connect to the api, we simply found out the backend's ip address, and then wrote something like the following. 

```python
import requests
backend_ip = 'http://192.168.99.100:5000'
response = requests.get(backend_ip)
response.json()
```

Now Kubernetes, we *could in theory* do the same thing.  This is because in Kubernetes each pod on each node given is also given it's own ip address.  Let's take through the `describe pods` command.

<img src="./ip-address-pod.png" width="70%">

> See that line at the bottom?  That is the ip address of the pod.

Ok, now let's see this in a diagram.  Below, we have two different nodes, and our API pods and Frontend pods just happen to be spread across these nodes.  As we can see, each pod is given a separate ip address.

<img src="./pod-ips-colors.jpg" width="70%">

Take a closer look at those ip addresses.  Notice that the pods in node 1 share similar ip addresses, as do the pods in node 2.  Only the last digit is different.  In Kubernetes, a pod's ip address is assigned in such a way that it also specify what node a given pod is on.  

This way, if we were to make a request to a particular ip address, kubernetes can find the node the pod is located on, and then go to that particular pod.

While we cannot make a request to any of these ip addresses from outside of the cluster, any pod in our cluster can make a request to any other pod through the ip address.

### But don't use IP addresses

So as we saw, Kubernetes *does* allow us to connect to a pod through ip addresses.  But as developers we'll avoid working with these pod ip addresses directly for a couple of reasons:

1. Remember that these pods can be destroyed and then recreated on a new node.  And with a new pod comes a new ip address.  So these ip addresses are not particularly stable - just like our pods aren't stable.  So we'll let our cluster keep track of all of the changing ip addresses.

2. Also, we'll often have *replicas* of our pods (as in a deployment).  So we don't want to manage *which particular pod* the request gets made to.  We'll let Kubernetes handle that.  

### Introducing Services

So instead of using the pod ip addresses directly, we'll enable Kubernetes to make a request to any of a set of pods.  This called a service.

> A **service** enables network access to a set of pods.

It does this by allowing us to map a single *virtual ip address* to a set of pods in the service.  So we'll make a request to a virtual ip address, and then the service will send us to one of the corresponding pods.   

<img src="./api-service.jpg" width="70%">

So with our service setup, we can have one of our frontend pods make a request to our service, and this request will then be passed along to one of the corresponding pods.

<img src="./domain-name.jpg" width="60%">

Now before we finish, notice one more thing.  Our service has an ip address, but it has also been given a domain name.  So our client does not need to make a request to the ip address, but instead can reference the domain name.  

In code, once we have our api-service setup, our frontend can make a request to it with something like the following.

```python
import requests
backend_ip = 'http://api-service:5000'
response = requests.get(api)
response.json()
```

### Summary

In this lesson, we learned how we can make a request from one pod to another.  As we saw, every pod in Kubernetes is given it's own ip address, and each pod has access to every other pod through the ip address.  But because these ip addresses are unstable, and because we are happy to make a request to any one of the pods in a deployment, we will not be using these ip addresses directly.  Instead we create a service, which gives us a virtual ip address and a domain name.  We'll make a request to a domain name of a service, and this request will be passed along to a particular pod.

### Resources

[Kubernetes Services Documentation](https://kubernetes.io/docs/concepts/services-networking/service/)

[Networking in Kubernetes](https://matthewpalmer.net/kubernetes-app-developer/articles/kubernetes-networking-guide-beginners.html)

[Inspecting Kubernetes Networking](https://www.digitalocean.com/community/tutorials/how-to-inspect-kubernetes-networking)

[Networking Medium Blog](https://medium.com/google-cloud/understanding-kubernetes-networking-services-f0cb48e4cc82)

[Kubeacademy](https://kube.academy/lessons/kube-proxy)